In [ ]:
model = joblib.load("lang_logreg_model.pkl")
vectorizer = joblib.load("lang_logreg_vectorizer.pkl")

In [ ]:
def smooth_predictions(results, strong_threshold=0.6, weak_threshold=0.05):
    raw_preds = []
    for token, lang_probs in results:
        if lang_probs:
            top_lang, top_prob = lang_probs[0]
            raw_preds.append(top_lang)
        else:
            raw_preds.append(None)

    # Second pass
    final_preds = []

    for i, (token, lang_probs) in enumerate(results):
        if lang_probs[0][1] >= strong_threshold:
            final_preds.append(lang_probs[0][0])
            continue

        # neighbor
        neighbor_langs = []

        if i > 0:
            neighbor_langs.append(raw_preds[i-1])
        if i < len(results)-1:
            neighbor_langs.append(raw_preds[i+1])

        neighbor_langs = [x for x in neighbor_langs if x is not None]

        resolved = None

        for nlang in neighbor_langs:
            for lang, prob in lang_probs:
                if lang == nlang and prob >= weak_threshold:
                    resolved = lang
                    break
            if resolved:
                break

        if resolved:
            final_preds.append(resolved)
        else:
            final_preds.append(lang_probs[0][0])

    return list(zip([t for t,_ in results], final_preds))


In [ ]:
def detect_words(text, threshold=0.10):
    words = text.split()

    X = vectorizer.transform(words)
    probs = model.predict_proba(X)
    classes = model.classes_

    results = []

    for i, word in enumerate(words):
        word_probs = []
        for lang, p in zip(classes, probs[i]):
            if p >= threshold:
                word_probs.append((lang, float(p)))

        word_probs.sort(key=lambda x: x[1], reverse=True)
        results.append((word, word_probs))

    return results

In [ ]:
def detect_with_smoothing(text):
    results = detect_words(text)
    smoothed = smooth_predictions(results)
    return smoothed

In [ ]:
text = "alii"
smoothed = detect_with_smoothing(text)

for token, lang in smoothed:
    print(f"{token:10} → {lang}")